In [264]:
using ArchGDAL
using StatsBase

In [388]:
coords = Array{Tuple{Float64, Float64}}(undef, 0)

ArchGDAL.registerdrivers() do
    ArchGDAL.read("data/gpw-v4-population-density-rev11_2020_2pt5_min_tif/gpw_v4_population_density_rev11_2020_2pt5_min.tif") do dataset

        # Get the band of data corresponding to population density per ~5km square of the Earth's surface
        band = ArchGDAL.getband(dataset, 1)
        min_lat = 8.040682
        max_lat = 11.2195684
        min_lon = -85.956896
        max_lon = -82.5060208
        
        
        band_width = ArchGDAL.width(band)
        band_height = ArchGDAL.height(band)        
        max_lat_ind = ceil((90 - min_lat) * band_height / 180)
        min_lat_ind = min(ceil((90 - max_lat) * band_height / 180), max_lat_ind - 1)
        min_lon_ind = ceil((min_lon + 180) * band_width / 360)
        max_lon_ind = max(ceil((max_lon + 180) * band_width / 360), min_lon_ind + 1)
        rows = UnitRange{Int}(min_lat_ind, max_lat_ind)
        cols = UnitRange{Int}(min_lon_ind, max_lon_ind)
        bounding_box_contents = ArchGDAL.read(band, rows, cols)
        bounding_box_contents = [i > 0 ? i : 0 for i in bounding_box_contents] # Have to clean up negative values
        
        # Convert indices into a set of values for an Empirical CDF
        # Convert population density into a set of weights for an Empirical CDF
        indices = collect(Iterators.flatten(1:length(bounding_box_contents)))
        population_density = collect(Iterators.flatten(bounding_box_contents))
        population_density /= sum(population_density)
        weights = FrequencyWeights(population_density)
        
        # Run the weighted sampling, to obtain the coordinates we will sample NSRDB from
        n = 1000
        samples = sample(indices, weights, n, replace=false)
    
        # Convert indices back to lat/long coordinates
        box_height, box_width = size(bounding_box_contents)
        
        function box_to_coords(index)
            x = div(index, box_width)
            y = index % box_width
            lat = min_lat + (max_lat - min_lat)/box_height*x
            lon = min_lon + (max_lon - min_lon)/box_width*y
            return (lat,lon)
        end
        
        for sample in samples
            push!(coords, box_to_coords(sample));
        end
    end
end

In [389]:
print(coords)

Tuple{Float64,Float64}[(9.26628, -84.1872), (9.34288, -84.4084), (9.30458, -83.5678), (9.34288, -83.7448), (9.34288, -83.4793), (9.18968, -85.0721), (9.57268, -85.6472), (9.26628, -84.2757), (9.30458, -84.5854), (9.30458, -84.0102), (9.22798, -84.4527), (8.76838, -83.6121), (8.92158, -83.0812), (9.34288, -83.6563), (8.27048, -84.2315), (9.22798, -85.3818), (9.41948, -83.0369), (9.91737, -83.6121), (9.26628, -84.0987), (9.34288, -83.789), (9.11308, -84.5854), (9.11308, -82.9927), (9.30458, -83.966), (9.30458, -84.0545), (9.18968, -84.9836), (9.18968, -83.6563), (9.15138, -84.3199), (10.5302, -82.7272), (9.30458, -83.9218), (9.15138, -85.2933), (9.38118, -83.3024), (8.69178, -85.426), (8.07898, -85.249), (9.07478, -85.0721), (9.26628, -83.0812), (9.22798, -84.4969), (9.38118, -83.5236), (8.30878, -83.5678), (8.23218, -83.0812), (10.7983, -84.7181), (9.22798, -84.9836), (9.38118, -83.3909), (8.80668, -83.8333), (9.53438, -83.7006), (9.34288, -84.143), (9.34288, -83.9218), (9.11308, -85.29

), (8.80668, -84.1872), (8.69178, -85.249), (10.0706, -85.6472), (8.88328, -83.5678), (9.61098, -85.7357), (9.30458, -82.9042), (8.61518, -85.8242), (9.34288, -83.5236), (9.64928, -85.8684), (8.46198, -85.0721), (9.30458, -84.2757), (9.61098, -85.3818), (9.11308, -85.6472), (9.87907, -84.3642), (10.9515, -84.3199), (8.73008, -85.249), (8.46198, -82.86), (8.84498, -83.3024), (9.11308, -85.9569), (9.91737, -83.3466), (9.18968, -84.4969), (9.41948, -82.9927), (9.07478, -82.5503), (8.76838, -84.3199), (10.4536, -83.0369), (9.95567, -83.4351), (8.38538, -85.9569), (9.99397, -83.2581), (9.07478, -84.4527), (8.76838, -83.789), (9.76417, -85.249), (8.15558, -83.789), (9.87907, -83.8775), (9.84077, -84.5854), (9.38118, -83.1697), (10.6834, -85.5145), (8.92158, -82.86), (8.04068, -84.2315), (8.73008, -85.2933), (9.34288, -85.426), (8.73008, -83.789), (9.38118, -84.8066), (9.53438, -85.1605), (10.1089, -85.8242), (8.27048, -83.0812), (8.76838, -84.0987), (10.7217, -85.2048), (10.0706, -82.7715), 

.80668, -84.143), (9.11308, -82.7272), (8.30878, -82.6387), (8.95988, -83.8333), (8.76838, -83.2139), (10.377, -83.5678), (9.80247, -83.4793), (8.84498, -84.143), (8.53858, -82.5945), (9.34288, -85.2048), (8.92158, -85.6914), (8.65348, -85.6914), (9.61098, -84.5854), (9.03648, -83.4793), (9.30458, -84.4084), (8.92158, -83.0369), (9.18968, -82.5945), (8.11728, -84.4969), (8.61518, -84.9836), (8.27048, -82.7272), (8.84498, -83.8333), (9.95567, -82.9927), (10.6834, -85.6472), (8.80668, -83.789), (8.88328, -83.2581), (8.23218, -83.1254), (8.92158, -85.5145), (8.84498, -84.8508), (9.03648, -85.5145), (10.2621, -85.0721), (9.72587, -83.7006), (8.34708, -82.9484), (8.61518, -84.5854), (8.53858, -85.426), (8.80668, -84.6296), (8.11728, -84.7624), (9.45778, -84.2757), (10.5685, -83.1697), (9.34288, -82.86), (9.03648, -85.6914), (8.99818, -83.6121), (8.50028, -85.7799), (8.50028, -85.9569), (9.03648, -83.3909), (10.3387, -83.7006), (9.53438, -82.6387), (9.34288, -82.9484), (9.91737, -82.5503), (